In [10]:
import os
from os.path import expanduser
home_dir = expanduser("~")
module_path = home_dir + '/code/modules/'
import sys
sys.path.append(module_path)
import time
import datetime
import importlib
import random
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import model_management
%load_ext autoreload
%autoreload 1
%aimport data_processing
%aimport plotting
%aimport keras_objects
%aimport pso
from data_processing import *
from plotting import *
from keras_objects import *
from pso import *

np.random.seed(999)
random.seed(999)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
### General parameters
run_on_cpu = False
total_set_size = 3000 # how many examples will be used for training+validation+testing
train_size = 1000
val_size = 1000
test_size = 1000
input_features = ['Halo_mass', 'Halo_mass_peak', 'Type', 'Concentration']
output_features = ['Stellar_mass', 'SFR']
nr_iters_before_restart_check = 60 # start making sure that the network did not converge to a local minimum
min_std_tol = 0.01 # minimum allowed std for any parameter
plot_threeD = 0
save_figs = 0
fig_dir = 'figures/'

### Network parameters
nr_hidden_layers = 10
activationFunction = 'tanh'
nr_neurons_per_layer = 10
loss_function = 'halo_mass_weighted_loss'

### PSO parameters
nIterations = 1000
nParticles = 40
xMin = -10
xMax = 10
alpha = 1
deltaT = 1
c1 = 2
c2 = 2
inertiaWeightStart = 1.4
inertiaWeightMin = 0.3
explorationFraction = 0.8

In [ ]:
if run_on_cpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
### Set name ending with parameters for figures to be saved
param_string = 'nLayers_%d_nNeurons_%d_actFun_%s_nTrainSamples_%d_nIterations_%d_' % (
    nr_hidden_layers, nr_neurons_per_layer, activationFunction, train_size, nIterations)
print(param_string)

# Load and preprocess the data

In [ ]:
# load the selected galaxyfile
galaxies, data_keys, unit_dict = load_galfile()
    
# prepare the training data
training_data_dict = divide_train_data(galaxies, data_keys, input_features, output_features, 
                                       int(total_set_size), int(train_size), int(val_size), int(test_size))
galaxies = None
training_data_dict = normalise_data(training_data_dict, norm)

In [3]:
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
galfile = pd.read_hdf('/scratch/data/galcats/P200/galaxies.Z01.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)

### Remove data points with halo mass below 10.5
print(np.shape(galaxies))
galaxies = galaxies[galaxies[:,6] > 10.5, :]
print(np.shape(galaxies))
print(galaxies[:10,6])

halo_min_mass = np.min(galaxies[:, 6])
halo_max_mass = np.max(galaxies[:, 6])

### Create the different data sets that will be used
n_data_points = galaxies.shape[0]
subset_indices = np.random.choice(n_data_points, total_set_size, replace=False)
train_indices = subset_indices[: train_size]
val_indices = subset_indices[train_size : train_size+val_size]
test_indices = subset_indices[train_size+val_size :]

x_train = np.zeros((len(train_indices), len(input_features)))
x_val = np.zeros((len(val_indices), len(input_features)))
x_test = np.zeros((len(test_indices), len(input_features)))
y_train = np.zeros((len(train_indices), len(output_features)))
y_val = np.zeros((len(val_indices), len(output_features)))
y_test = np.zeros((len(test_indices), len(output_features)))

for i in range(len(input_features)):
    x_train[:,i] = galaxies[train_indices, data_dict[input_features[i]]]
    x_val[:,i] = galaxies[val_indices, data_dict[input_features[i]]]
    x_test[:,i] = galaxies[test_indices, data_dict[input_features[i]]]
    
for i in range(len(output_features)):
    y_train[:,i] = galaxies[train_indices, data_dict[output_features[i]]]
    y_val[:,i] = galaxies[val_indices, data_dict[output_features[i]]]
    y_test[:,i] = galaxies[test_indices, data_dict[output_features[i]]]

### If you want to preprocess the data

for i in range(np.size(x_train, 1)):
    x_data_means = np.mean(x_train, 0)
    x_data_stds = np.std(x_train, 0)

    x_train_norm = (x_train - x_data_means) / x_data_stds
    x_val_norm = (x_val - x_data_means) / x_data_stds
    x_test_norm = (x_test - x_data_means) / x_data_stds

for i in range(np.size(y_train, 1)):
    y_data_means = np.mean(y_train, 0)
    y_data_stds = np.std(y_train, 0)

    y_train_norm = (y_train - y_data_means) / y_data_stds
    y_val_norm = (y_val - y_data_means) / y_data_stds
    y_test_norm = (y_test - y_data_means) / y_data_stds

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
(594946, 20)
(306925, 20)
[11.06128  10.74806  10.517851 11.935673 11.820144 11.564987 10.759135
 10.828308 10.803138 10.55098 ]


In [6]:
print(np.mean(y_test_norm, 0))
print(np.std(y_test_norm, 0))

[-0.00493404 -0.02780973]
[1.03360238 0.88024908]


In [7]:
### Get a feel for the data
for i in range(len(input_features)):
    print(input_features[i],': min: %.2e, max: %.2e.' % (np.min(x_train[:,i]), np.max(x_train[:,i])))
for i in range(len(output_features)):
    print(output_features[i],': min: %.2e, max: %.2e.' % (np.min(y_train[:,i]), np.max(y_train[:,i])))

Halo_mass : min: 1.05e+01, max: 1.41e+01.
Halo_mass_peak : min: 1.05e+01, max: 1.41e+01.
Type : min: 0.00e+00, max: 2.00e+00.
Concentration : min: 1.01e+00, max: 8.28e+02.
Stellar_mass : min: 7.00e+00, max: 1.16e+01.
SFR : min: 0.00e+00, max: 8.82e+00.


In [ ]:
### Visualisation for when we have 2 input features
%matplotlib notebook
input_feat_1 = 0
input_feat_2 = 1
output_feat = 1

fig = plt.figure(1, figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_train_norm[:500,input_feat_1], x_train_norm[:500,input_feat_2], 
           y_train_norm[:500,output_feat])
ax.set_xlabel('%s log($M_{H}/M_{S}$)' % (input_features[input_feat_1]))
ax.set_ylabel('%s log($M_{H}/M_{S}$)' % (input_features[input_feat_2]))
ax.set_zlabel('%s log($M_{G}/M_{S}$)' % (output_features[output_feat]))
plt.show()

# Create a new network

In [ ]:
network = Feed_Forward_Neural_Network(nr_hidden_layers, nr_neurons_per_layer, input_features, output_features, 
                                      activationFunction)
network.pso_setup({'inertiaWeightMin': 0.1})
network.pso_train(nIterations, training_data_dict, loss_function, speed_check=False)

In [12]:
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
### OLD SHITTT DONT USE
network = Feed_Forward_Neural_Network(nr_hidden_layers, nr_neurons_per_layer, input_features, output_features, 
                                      activationFunction)
network.pso_setup({'inertiaWeightMin': 0.1})
network.pso_train(nIterations, x_train_norm, y_train_norm, x_val_norm, y_val_norm, data_dict, speed_check=False)


Iteration 0, particle 0, new swarm best. Train: 482.630, Val: 482.388
Iteration 0, particle 1, new swarm best. Train: 199.518, Val: 200.337
Iteration 0, particle 2, new swarm best. Train: 181.035, Val: 167.628
Iteration 0, particle 15, new swarm best. Train: 107.157, Val: 111.159
Iteration 2, particle 1, new swarm best. Train: 83.046, Val: 79.279
Iteration 2, particle 38, new swarm best. Train: 75.747, Val: 77.890
Iteration 3, particle 22, new swarm best. Train: 75.695, Val: 74.420
Iteration 3, particle 28, new swarm best. Train: 44.008, Val: 45.939
Iteration 5, particle 28, new swarm best. Train: 41.107, Val: 39.798
Iteration 6, particle 10, new swarm best. Train: 37.852, Val: 38.062
Iteration 6, particle 33, new swarm best. Train: 34.302, Val: 36.214
Iteration 8, particle 12, new swarm best. Train: 25.685, Val: 25.408
Iteration 9, particle 20, new swarm best. Train: 17.032, Val: 18.317
Iteration 9, particle 22, new swarm best. Train: 16.049, Val: 16.624
Iteration 9, particle 29, new 

NameError: name 'nr_iters_before_restart_check' is not defined

In [ ]:
### Visualisation of prediction strength for when we have 2 input features
if plot_threeD and len(input_features) == 2:
    predictedY = PredictFunc(bestWeightList, bestBiasList, model, 'test')
    fig = plt.figure(2, figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_test[:,0], x_test[:,1], 
               y_test[:,0], s=3)
    ax.scatter(x_test[:,0], x_test[:,1], 
               predictedY, s=3)
    ax.set_xlabel('%s log($M_{H}/M_{S}$)' % (input_features[0]))
    ax.set_ylabel('%s log($M_{H}/M_{S}$)' % (input_features[1]))
    ax.set_zlabel('%s log($M_{G}/M_{S}$)' % (output_features[0]))

In [ ]:
%matplotlib inline
nBins = 8
bin_edges = np.linspace(halo_min_mass, halo_max_mass, nBins+1)

predictedY = model.predict(x_test)

for i, feat in enumerate(output_features):
    
    
    ### Plot 1
    fig = plt.figure(figsize=(16,16))
    ax = plt.subplot(211)
    plt.plot(y_test[:,i], y_test[:,i], 'k.')
    plt.plot(predictedY[:,i], y_test[:,i], 'g.')
    plt.ylabel('True %s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.xlabel('Predicted %s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.legend(['Ideal result', 'predicted ' + feat], loc='upper center')
    plt.title('nIterations: %d, training set size: %d, test mse score: %.2e\n' % (nIterations, 
        train_size, testScore) + 
        '%d input feature(s): [%s]\n%d output feature(s): [%s]\n%d test data points (test) shown' % (
        len(input_features), ', '.join(input_features), len(output_features), ', '.join(output_features),
        test_size), y=1.03, fontsize=20)
    plt.show
        
    if save_figs:
        fig.savefig(fig_dir+'pso_output_scatter_%d_plot_from_' % (i+1)+'_and_'.join(input_features)+'_to_'+
            '_and_'.join(output_features)+'_with_'+param_string+'.png', bbox_inches = 'tight')
    
    ### Plot 2 - boxplot
    
    # bin_means contain (0: mean of the binned values, 1: bin edges, 2: numbers pointing each example to a bin)
    bin_means_true = stats.binned_statistic(x_test[:,i], y_test[:,i], bins=bin_edges)
    bin_means_pred = stats.binned_statistic(x_test[:,i], predictedY[:,i].flatten(), bins=bin_edges)
    bin_centers = []
    for iBin in range(nBins):
        bin_centers.append((bin_means_true[1][iBin] + bin_means_true[1][iBin+1]) / 2)
    sorted_true_y_data = []
    sorted_pred_y_data = []
    for iBin in range(1,nBins+1):
        sorted_true_y_data.append(y_test[bin_means_true[2] == iBin, i])
        sorted_pred_y_data.append(predictedY[bin_means_pred[2] == iBin,i])
    
    fig = plt.figure(figsize=(16,8))
    ax = plt.subplot(212)

    bin_pos = np.array([-2,-1]) # (because this makes it work)
    x_label_centers = []
    for iBin in range(nBins):
        # Every boxplot adds 2 boxes, one from the true data and one from the predicted data
        bin_pos += 3 
        plt.boxplot([sorted_true_y_data[iBin], sorted_pred_y_data[iBin]] , positions = bin_pos, widths = 0.9)
        x_label_centers.append(np.mean(bin_pos))
    
    plt.ylabel('%s %s' % (feat, unit_dict[feat]), fontsize=15)
    plt.xlabel('True Halo mass log($M_{G}/M_{S}$)', fontsize=15)
    ax.set_xlim(left=x_label_centers[0]-2, right=x_label_centers[-1]+2)
    #xlim(0,bin_pos[1] + 1)
    plt.xticks(x_label_centers, bin_centers) TODO fixa siffrorna
    plt.text(12,7,'Left: true data. Right: predicted data.', fontsize=20)
    
    if feat == 'SFR':
        ax.axhline(y=0, linestyle='--')
    
    #plt.title('nIterations: %d, training set size: %d, test mse score: %.2e\n' % (nIterations, 
    #    train_size, testScore) + 
    #    '%d input feature(s): [%s]\n%d output feature(s): [%s]\n%d test data points (test) shown' % (
    #    len(input_features), ', '.join(input_features), len(output_features), ', '.join(output_features),
    #    test_size), y=1.03, fontsize=20)
    
    plt.show()
    
    if save_figs:
        fig.savefig(fig_dir+'pso_output_boxplot_%d_from_' % (i+1)+'_and_'.join(input_features)+'_to_'+
            '_and_'.join(output_features)+'_with_'+param_string+'.png', bbox_inches = 'tight')

In [ ]:
# Summarize history for loss
%matplotlib inline
fig = plt.figure(5, figsize=(8,8))
plt.plot(trainingScoreHistory, 'b')
plt.plot(validationScoreHistory, 'r')
plt.yscale('log')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()